In [1]:
import autograd.numpy as np
from autograd import grad

In [2]:
class ORNeuron():
    def __init__(self, n):
        self.n = n
        self.weights = np.random.rand(n)
        
    def getWeights(self):
        return self.weights    
    
    def present(self, inputs):
        z = np.sum(np.multiply(inputs, self.weights))
        return 1 - np.power(np.e, -z)
    
    def updateWeights(self, g):
        self.weights -= g
        
        for i in range(0, len(self.weights)):
            if self.weights[i] < 0:
                self.weights[i] = 0
        
    def setGrad(self, g):
        self.grad = g
        
    def applyGrad(self):
        self.updateWeights(self.grad)
        self.grad = None
        
    def __repr__(self):
        return str(self.weights[1:len(self.weights)])

In [3]:
class ANDNeuron():
    def __init__(self, n):
        self.weights = np.random.rand(n)
        
    def getWeights(self):
        return self.weights
        
    def present(self, inputs):
        i = 1.0 - inputs
        i[0] = 1.0
        
        z = np.sum(np.multiply(i, self.weights))
        return np.power(np.e, -z)
    
    def updateWeights(self, g):
        self.weights -= g
        
        for i in range(0, len(self.weights)):
            if self.weights[i] < 0:
                self.weights[i] = 0
        
    def setGrad(self, g):
        self.grad = g
        
    def applyGrad(self):
        self.updateWeights(self.grad)
        self.grad = None
        
    def __repr__(self):
        return str(self.weights[1:len(self.weights)])

In [4]:
class CNFNetwork():
    def __init__(self, n, c):
        self.n = n
        self.c = c
        
        d = []
        for i in range(0, c):
            d.append(ORNeuron(n*2 + 1))
        
        self.disjunctions = np.array(d)
        
        if not c == 0:
            self.conjunction = ANDNeuron(c + 1)
        else:
            self.conjunction = ANDNeuron(n*2 + 1)
        
    def getDisjunctions(self):
        return self.disjunctions
    
    def getConjunction(self):
        return self.conjunction
    
    def fowardprop(self, inputs):
        actualIn = self.__convertInputs__(inputs)
        
        if not self.c == 0:
            dout = [1]
            for d in self.disjunctions:
                dout.append(d.present(actualIn))
            
            actualIn = dout
            
        return self.conjunction.present(np.array(actualIn))
    
    def __convertInputs__(self, inputs):
        actual = [1]
        
        for i in inputs:
            actual.append(i)
            actual.append(1-i)
        
        return np.array(actual)
    
    def __repr__(self):
        s = "Disjunctions -> "
        for d in self.disjunctions:
            s += (str(d) + ", ")
            
        s += ("\nConjunction -> " + str(self.conjunction) + "\n")
            
        return s

In [5]:
def SSE(network, data, targets, p=False):
    predictions = predict(network, data)
    
#     if p:
#         print(predictions)
    
    return np.sum(np.power(np.subtract(targets, predictions), 2.0))
    
def predict(network, data):
    t = np.array([network.fowardprop(d) for d in data])
    return t
    

In [6]:
def trainCNFNetwork(data, targets, inputNodes, numC, it=10000, lr=0.1):
    network = CNFNetwork(inputNodes, numC)
    print(network)
    print("Initial Loss: ", SSE(network, data, targets, True))
    
    pterb = 0.0001
    
    for i in range(1, it):
        if i%1000 == 0:
            print("Iteration -> " + str(i) + " : " + str(SSE(network, data, targets)))
            
        for d in network.getDisjunctions():
            g = computeNeuronGrad(network, d, pterb, data, targets)
            d.setGrad(g * lr)
            
        g = computeNeuronGrad(network, network.getConjunction(), pterb, data, targets)
        
        network.getConjunction().updateWeights(g * lr)
        for d in network.getDisjunctions():
            d.applyGrad()
            
    print("Trained Loss: ", SSE(network, data, targets, True))
    return network

def computeNeuronGrad(network, neuron, pterb, data, targets):
    gradient = np.zeros(len(neuron.getWeights()))
    for k in range(0, len(neuron.getWeights())):
        g = np.zeros(len(neuron.getWeights()))
        g[k] = -pterb

        oldSSE = SSE(network, data, targets)
        neuron.updateWeights(g)
        newSSE = SSE(network, data, targets)
        neuron.updateWeights(-g)
                
        gradient[k] = (newSSE - oldSSE)/pterb
        
    return gradient

# NOT

In [7]:
dataNOT = np.array([[1.0], [0.0]])
targetsNOT = np.array([0.0, 1.0])
trainCNFNetwork(dataNOT, targetsNOT, 1, 0)

Disjunctions -> 
Conjunction -> [ 0.27908806  0.43288811]

Initial Loss:  0.460428859876
Iteration -> 1000 : 0.00256161360801
Iteration -> 2000 : 0.00126726389852
Iteration -> 3000 : 0.000841496123815
Iteration -> 4000 : 0.000629788885764
Iteration -> 5000 : 0.000503163604421
Iteration -> 6000 : 0.00041892004715
Iteration -> 7000 : 0.000358833724639
Iteration -> 8000 : 0.000313818184571
Iteration -> 9000 : 0.000278835903905
Trained Loss:  0.000250869217382


Disjunctions -> 
Conjunction -> [ 0.          4.14516891]

# AND
For this we dont neeed any disjunctions

In [8]:
dataAND = np.array([[0.0, 0.0], [0.0, 1.0], [1.0, 0.0], [1.0, 1.0]])
targetsAND = np.array([0.0, 0.0, 0.0, 1.0])
trainCNFNetwork(dataAND, targetsAND, 2, 0)

Disjunctions -> 
Conjunction -> [ 0.09729661  0.40437016  0.59076866  0.74971913]

Initial Loss:  1.02939324302
Iteration -> 1000 : 0.00513952254646
Iteration -> 2000 : 0.00254075061141
Iteration -> 3000 : 0.00168631145204
Iteration -> 4000 : 0.00126166488227
Iteration -> 5000 : 0.00100777184534
Iteration -> 6000 : 0.000838904869129
Iteration -> 7000 : 0.00071848780167
Iteration -> 8000 : 0.000628289567814
Iteration -> 9000 : 0.000558205313526
Trained Loss:  0.000502183092491


Disjunctions -> 
Conjunction -> [ 4.14414572  0.          4.14517597  0.        ]

# OR

In [9]:
dataOR = np.array([[0.0, 0.0], [0.0, 1.0], [1.0, 0.0], [1.0, 1.0]])
targetsOR = np.array([0.0, 1.0, 1.0, 1.0])
trainCNFNetwork(dataOR, targetsOR, 2, 1)

Disjunctions -> [ 0.03597072  0.54622006  0.28498513  0.46150847], 
Conjunction -> [ 0.91897467]

Initial Loss:  0.859634250643
Iteration -> 1000 : 0.0147264514446
Iteration -> 2000 : 0.00665424999811
Iteration -> 3000 : 0.00422199561736
Iteration -> 4000 : 0.00306961003498
Iteration -> 5000 : 0.00240224827008
Iteration -> 6000 : 0.00196868011984
Iteration -> 7000 : 0.00166512071001
Iteration -> 8000 : 0.0014410967145
Iteration -> 9000 : 0.0012691786224
Trained Loss:  0.00113320461181


Disjunctions -> [ 5.33022     0.          5.33026365  0.        ], 
Conjunction -> [ 3.85379597]

# NAND

In [10]:
dataNAND = np.array([[0.0, 0.0], [0.0, 1.0], [1.0, 0.0], [1.0, 1.0]])
targetsNAND = np.array([1.0, 1.0, 1.0, 0.0])
trainCNFNetwork(dataNAND, targetsNAND, 2, 1)

Disjunctions -> [ 0.2670529   0.71459223  0.52982119  0.41614743], 
Conjunction -> [ 0.24131879]

Initial Loss:  0.736507950989
Iteration -> 1000 : 0.0145598993122
Iteration -> 2000 : 0.00661740881009
Iteration -> 3000 : 0.00420662032181
Iteration -> 4000 : 0.00306130179544
Iteration -> 5000 : 0.00239708104691
Iteration -> 6000 : 0.00196516924452
Iteration -> 7000 : 0.00166258581079
Iteration -> 8000 : 0.00143918357024
Iteration -> 9000 : 0.00126768519266
Trained Loss:  0.00113200746639


Disjunctions -> [ 0.          5.33094093  0.          5.33087605], 
Conjunction -> [ 3.85433414]

# NOR

In [11]:
dataNAND = np.array([[0.0, 0.0], [0.0, 1.0], [1.0, 0.0], [1.0, 1.0]])
targetsNAND = np.array([1.0, 0.0, 0.0, 0.0])
trainCNFNetwork(dataNAND, targetsNAND, 2, 2)

Disjunctions -> [ 0.6713813   0.45593178  0.11943221  0.55086449], [ 0.04325333  0.15126712  0.38242037  0.26897606], 
Conjunction -> [ 0.04322681  0.54394268]

Initial Loss:  0.820470053067
Iteration -> 1000 : 0.0241405118157
Iteration -> 2000 : 0.00861527176654
Iteration -> 3000 : 0.00529972730603
Iteration -> 4000 : 0.00382827656174
Iteration -> 5000 : 0.00299095435523
Iteration -> 6000 : 0.00245027690704
Iteration -> 7000 : 0.00207263608272
Iteration -> 8000 : 0.00179421311504
Iteration -> 9000 : 0.00158062496467
Trained Loss:  0.00141170429587


Disjunctions -> [ 0.          0.          0.          5.66369549], [ 0.          5.70631119  0.          0.        ], 
Conjunction -> [ 3.94126039  3.98088195]

# XOR

In [12]:
dataNAND = np.array([[0.0, 0.0], [0.0, 1.0], [1.0, 0.0], [1.0, 1.0]])
targetsNAND = np.array([0.0, 1.0, 1.0, 0.0])
trainCNFNetwork(dataNAND, targetsNAND, 2, 2)

Disjunctions -> [ 0.23628204  0.98037393  0.77622604  0.38387031], [ 0.00459108  0.37895811  0.05353995  0.40983986], 
Conjunction -> [ 0.21337505  0.6377396 ]

Initial Loss:  1.03906513532
Iteration -> 1000 : 0.716665358151
Iteration -> 2000 : 0.686879187644
Iteration -> 3000 : 0.679023421963
Iteration -> 4000 : 0.675493209024
Iteration -> 5000 : 0.673505165411
Iteration -> 6000 : 0.672235507916
Iteration -> 7000 : 0.671356701026
Iteration -> 8000 : 0.670713415296
Iteration -> 9000 : 0.670222729861
Trained Loss:  0.66983643155


Disjunctions -> [ 0.30954545  0.93013653  0.75667035  0.42645202], [ 0.          7.05349     0.          7.05349383], 
Conjunction -> [ 0.          3.24415127]

# IMPLIES

In [13]:
dataIMPLYS = np.array([[0.0, 0.0], [0.0, 1.0], [1.0, 0.0], [1.0, 1.0]])
targetsIMPLYS = np.array([1.0, 1.0, 0.0, 1.0])
trainCNFNetwork(dataIMPLYS, targetsIMPLYS, 2, 1)

Disjunctions -> [ 0.66001029  0.21698503  0.21025963  0.27813663], 
Conjunction -> [ 0.64862298]

Initial Loss:  0.852875890091
Iteration -> 1000 : 0.0148887839343
Iteration -> 2000 : 0.00668978603467
Iteration -> 3000 : 0.00423677449114
Iteration -> 4000 : 0.00307758211592
Iteration -> 5000 : 0.00240720127841
Iteration -> 6000 : 0.00197204312843
Iteration -> 7000 : 0.00166754765698
Iteration -> 8000 : 0.00144292771334
Iteration -> 9000 : 0.0012706075217
Trained Loss:  0.00113434976913


Disjunctions -> [ 0.          5.32961667  5.32959321  0.        ], 
Conjunction -> [ 3.85328125]

# Complicated Formula

## (a AND b) OR (NOT c)
CNF: (a OR (NOT c)) AND (b OR (NOT c))

In [8]:
dataInteresting1 = np.array([[0.0, 0.0, 0.0], [0.0, 0.0, 1.0], [0.0, 1.0, 0.0], [0.0, 1.0, 1.0], [1.0, 0.0, 0.0], [1.0, 0.0, 1.0], [1.0, 1.0, 0.0], [1.0, 1.0, 1.0]])
targetsInteresting1 = np.array([1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0])
trainCNFNetwork(dataInteresting1, targetsInteresting1, 3, 2)

Disjunctions -> [ 0.67922307  0.58791759  0.3346763   0.46705493  0.72680952  0.99853502], [ 0.30379515  0.61549913  0.36631291  0.05389693  0.20947085  0.54337965], 
Conjunction -> [ 0.02244506  0.94389352]

Initial Loss:  1.93106186702
Iteration -> 1000 : 0.205259518578
Iteration -> 2000 : 0.161838896882
Iteration -> 3000 : 0.140655258073
Iteration -> 4000 : 0.127272742999
Iteration -> 5000 : 0.117746853722


KeyboardInterrupt: 

## NOT ( a AND b AND c)
CNF: (NOT a) OR (NOT b) OR (NOT c)

In [8]:
dataInteresting2 = np.array([[0.0, 0.0, 0.0], [0.0, 0.0, 1.0], [0.0, 1.0, 0.0], [0.0, 1.0, 1.0], [1.0, 0.0, 0.0], [1.0, 0.0, 1.0], [1.0, 1.0, 0.0], [1.0, 1.0, 1.0]])
targetsInteresting2 = np.array([1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0])
trainCNFNetwork(dataInteresting2, targetsInteresting2, 3, 1)

Disjunctions -> [ 0.77248135  0.91413476  0.0594183   0.9376519   0.87540569  0.57473465], 
Conjunction -> [ 0.25904396]

Initial Loss:  1.29383934054
Iteration -> 1000 : 0.0215283049907
Iteration -> 2000 : 0.00961174395394
Iteration -> 3000 : 0.00605553499049
Iteration -> 4000 : 0.00438216473351
Iteration -> 5000 : 0.00341786440018
Iteration -> 6000 : 0.00279372695655
Iteration -> 7000 : 0.00235803947172
Iteration -> 8000 : 0.00203728855623
Iteration -> 9000 : 0.00179164432472
Trained Loss:  0.00159769852035


Disjunctions -> [ 0.          5.29446248  0.          5.29441863  0.          5.29435214], 
Conjunction -> [ 3.74745824]

## a AND b AND c

In [9]:
dataInteresting3 = np.array([[0.0, 0.0, 0.0], [0.0, 0.0, 1.0], [0.0, 1.0, 0.0], [0.0, 1.0, 1.0], [1.0, 0.0, 0.0], [1.0, 0.0, 1.0], [1.0, 1.0, 0.0], [1.0, 1.0, 1.0]])
targetsInteresting3 = np.array([0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0])
trainCNFNetwork(dataInteresting3, targetsInteresting3, 3, 3)

Disjunctions -> [ 0.50594142  0.53135592  0.13034028  0.70396741  0.64741553  0.116168  ], [ 0.97850827  0.36923505  0.805332    0.07594967  0.21654293  0.2737607 ], [ 0.47683372  0.99139624  0.19557071  0.92410033  0.15578902  0.45872944], 
Conjunction -> [ 0.18335982  0.84833336  0.67238309]

Initial Loss:  3.2606695219
Iteration -> 1000 : 0.23496313411
Iteration -> 2000 : 0.174289282207
Iteration -> 3000 : 0.147792047189
Iteration -> 4000 : 0.132030993345
Iteration -> 5000 : 0.121251114898
Iteration -> 6000 : 0.113218858185
Iteration -> 7000 : 0.106899946293
Iteration -> 8000 : 0.101742301174
Iteration -> 9000 : 0.0974178965214
Trained Loss:  0.093716954439


Disjunctions -> [ 0.        0.        0.        0.        6.664869  0.      ], [ 2.04620247  0.          2.04620247  0.          0.          0.        ], [ 0.96734099  0.72825114  0.63849275  0.70854046  0.65668107  0.18519956], 
Conjunction -> [  3.68741218  14.70749228   0.        ]

# (a XOR b) AND c
CNF: ((NOT a) OR (NOT b)) AND (a OR b) AND c

In [10]:
dataInteresting4 = np.array([[0.0, 0.0, 0.0], [0.0, 0.0, 1.0], [0.0, 1.0, 0.0], [0.0, 1.0, 1.0], [1.0, 0.0, 0.0], [1.0, 0.0, 1.0], [1.0, 1.0, 0.0], [1.0, 1.0, 1.0]])
targetsInteresting4 = np.array([0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0])
trainCNFNetwork(dataInteresting4, targetsInteresting4, 3, 3)

Disjunctions -> [ 0.79632812  0.68701815  0.15567894  0.02820487  0.67215438  0.55405899], [ 0.94541744  0.75253432  0.2531231   0.45607826  0.15366458  0.7868916 ], [ 0.9898861   0.36346758  0.99991556  0.46991022  0.72938421  0.88039021], 
Conjunction -> [ 0.53691676  0.5018143   0.63306207]

Initial Loss:  2.11939845615
Iteration -> 1000 : 0.366337792068
Iteration -> 2000 : 0.256110678263
Iteration -> 3000 : 0.214483150848
Iteration -> 4000 : 0.190607210586
Iteration -> 5000 : 0.174465999654
Iteration -> 6000 : 0.162538732249
Iteration -> 7000 : 0.153218572536
Iteration -> 8000 : 0.145651209374
Iteration -> 9000 : 0.139331836401
Trained Loss:  0.13394030331


Disjunctions -> [ 0.          2.11023089  0.          2.11023122  2.36002622  0.        ], [ 0.71005122  1.11435871  0.16068307  0.67497645  0.54090159  0.52611276], [ 6.85682635  0.          6.85689923  0.          0.          0.        ], 
Conjunction -> [ 17.53234172   0.           3.95484957]